In [105]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor

In [106]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; ChessDataBot/1.0)"
}

def get_archives(username):
    url = f"https://api.chess.com/pub/player/{username}/games/archives"
    response = requests.get(url, headers=HEADERS)

    if response.status_code != 200:
        raise Exception(f"Erreur API archives: {response.status_code}")

    return response.json()["archives"]


In [107]:
def get_player_country(username):
    if not username:
        return None

    try:
        url = f"https://api.chess.com/pub/player/{username}"
        r = requests.get(url, headers=HEADERS, timeout=10)

        if r.status_code != 200:
            return None

        data = r.json()
        country_url = data.get("country")

        if not country_url:
            return None

        return country_url.split("/")[-1]

    except Exception:
        return None

In [108]:
def get_games_from_archive(archive_url):
    response = requests.get(archive_url, headers=HEADERS)

    if response.status_code != 200:
        raise Exception(f"Erreur API games: {response.status_code}")

    return response.json()["games"]


In [109]:
def parse_games(games):
    table = []

    for game in games:
        row = {
            "uuid": game.get("uuid"),
            "url": game.get("url"),
            "white": game["white"]["username"],
            "black": game["black"]["username"],
            "result_white": game["white"]["result"],
            "result_black": game["black"]["result"],
            "white_elo": game["white"]["rating"],
            "black_elo": game["black"]["rating"],
            "time_control": game.get("time_control"),
            "time_class": game.get("time_class"),
            "rated": game.get("rated"),
            "eco": game.get("eco"),
            "accuracy_white": game.get("accuracies", {}).get("white"),
            "accuracy_black": game.get("accuracies", {}).get("black"),
            "pgn": game.get("pgn"),
            "end_time": game.get("end_time")
        }

        table.append(row)

    return table


In [110]:
username = "ahmedbdk"
username =username.lower()
# 1. récupérer les archives
archives = get_archives(username)

games = []

# 2. Parcourir toutes les archives
for archive in archives:
    # 3. récupérer les parties
    games += get_games_from_archive(archive)
    

# 4. parser les parties dans un tableau
table = parse_games(games)

# 5. résultat
print(f"{len(table)} parties récupérées")


3168 parties récupérées


In [111]:
bronze_df = pd.DataFrame(table)

In [112]:
silver_df = bronze_df.copy()

In [113]:
silver_df['white'] = silver_df['white'].str.lower()
silver_df['black'] = silver_df['black'].str.lower()
silver_df[['accuracy_white','accuracy_black']] = silver_df[['accuracy_white','accuracy_black']].fillna('None')
silver_df['end_datetime'] = pd.to_datetime(bronze_df['end_time'], unit='s')
silver_df['rated'] = silver_df['rated'].astype(bool)
silver_df[['white_elo','black_elo']] = silver_df[['white_elo','black_elo']].astype(int)
silver_df = silver_df.drop_duplicates(subset='uuid')

In [114]:
gold_df = silver_df.copy()

In [115]:
gold_df['player'] = username
gold_df['opponent'] = np.where (
    (username == gold_df ['white']),
     gold_df['black'],
     gold_df['white'])
gold_df['result'] = np.where (
    (username == gold_df['white']),
    gold_df['result_white'],
    gold_df['result_black'])
gold_df['player_elo'] = np.where (
    (username == gold_df ['white']),
    gold_df['white_elo'],
    gold_df['black_elo'])
gold_df['opponent_elo'] = np.where (
    (username == gold_df ['white']),
    gold_df['black_elo'],
    gold_df['white_elo'])
gold_df['player_color'] = np.where (
    (username == gold_df['white']),
    'white',
    'black')
gold_df['opponent_color'] = np.where (
    (username == gold_df['white']),
    'black',
    'white')
gold_df['player_accuracy'] = np.where (
    (username == gold_df['white']),
    gold_df['accuracy_white'],
    gold_df['accuracy_black'])
gold_df['opponent_accuracy'] = np.where (
    (username == gold_df['white']),
    gold_df['accuracy_black'],
    gold_df['accuracy_white'])

gold_df['opening'] = gold_df['eco'].str.split('openings/').str[1]



In [116]:
new_order = ['uuid','player','player_elo','opponent','opponent_elo','player_color','opponent_color','result','time_class','time_control','opening','rated','end_datetime','player_accuracy','opponent_accuracy','url']
gold_df = gold_df[new_order]

In [117]:
unique_opponents = gold_df['opponent'].dropna().unique()
len(unique_opponents)

3023

In [118]:
opponent_country_map = {}

with ThreadPoolExecutor(max_workers=15) as executor:
    results = executor.map(get_player_country, unique_opponents)

    for opponent, country in tqdm(
        zip(unique_opponents, results),
        total=len(unique_opponents),
        desc="Fetching opponent countries"
    ):
        opponent_country_map[opponent] = country


Fetching opponent countries: 100%|█████████████████████████████████████████████████| 3023/3023 [15:27<00:00,  3.26it/s]


In [119]:
opponent_country_map

{'houssemiladi': 'TN',
 'danyagggy': 'KZ',
 'dalleji4real': 'FR',
 'stilllearning81': 'US',
 'azzipi': 'TH',
 'piyushbhatia2': 'IN',
 'mneimar1': 'MA',
 'valle546': 'DE',
 'bolagato3': 'BR',
 'fco_lazcano-inactive': 'CL',
 'zimians': 'ZA',
 'selvadarshu': 'MY',
 'vyom1401': 'SE',
 'jugobetrugo1223': 'DE',
 'peterbennett04': 'GB',
 'tiwana1777': 'CA',
 '5439hfkfo': 'CN',
 'chess12345612345': 'GB',
 'toomag': 'IR',
 'ravenous_demon1134': 'US',
 'shorouk02': 'IT',
 'lyphu': 'VN',
 'chessearner555': 'IN',
 'riderp82': 'IN',
 'kchessboard': 'DZ',
 'eavvbb': 'IT',
 'shonathebest': 'CA',
 'miagame': 'KP',
 'mayar9998': 'SA',
 'mmaalik01': 'CA',
 'gagimirka': 'RS',
 'patrikfelix': 'SK',
 'dr7yaya': 'IN',
 'zurye': 'MX',
 'mrhamidrezaa': 'IR',
 'fisu1330': 'UY',
 'geedi23': 'SE',
 'tatan9402': 'CO',
 'hiradanish': 'US',
 'phoenixak47': 'BD',
 'iwilldefeatadi': 'IN',
 'ice_tain': 'KR',
 'f1kj5n1': 'SG',
 'virunbhai94': 'IN',
 'nerthee': 'GB',
 'perejoan03': 'ES',
 'crowomagnifico': 'BR',
 'sierr

In [120]:
gold_df['opponent_country'] = gold_df['opponent'].map(opponent_country_map)
gold_df[gold_df['opponent_country'].isna()]

,uuid,player,player_elo,opponent,opponent_elo,player_color,opponent_color,result,time_class,time_control,opening,rated,end_datetime,player_accuracy,opponent_accuracy,url,opponent_country


In [121]:
output_path = f"chess_games_{username}_gold.csv"

gold_df.to_csv(
    output_path,
    index=False,
    encoding="utf-8"
)


In [122]:
archives[-1]

'https://api.chess.com/pub/player/ahmedbdk/games/2025/09'